## Retrieving Relevant Context using Word Embeddings

### Work Breakdown

- [x] Find pre-trained embeddings in Bulgarian
- [x] Load wikipedia documents
- [x] Chunk documents into sentences
    - [ ] Maybe it would be better to start with passing full documents, in order to keep the context?
- [x] Load embedding vectors
    - [x] Build an index from word to vector for mapping words to vectors
    - [x] Build an index from vector to vector for nearest neighbour search
- [x] Map each sentence/passage to an embedding vector
    - [ ] Paremeterize the aggregation function - avg, max, etc.
- [x] Load questions and answer options from `bg_rc-v1.0.json`
- [x] For each question find the nearest contexts (passages, sentences)
- [x] For each question add to it the list of most relevant texts to the questions and answers under the "context" key
- [ ] Paremeterize each step and expose configuration for grid search

In [8]:
import json
import numpy as np
import pandas as pd
import ujson
import glob
import ngtpy
import codecs

from polyglot.text import Text, Word

from tqdm.auto import tqdm

from IPython.utils import io

In [6]:
tqdm.pandas()

In [2]:
config = {
    'documents_glob': '../data/context/bgwiki.json/*/wiki_*',
    'embeddings_path': '../data/word2vec/model.txt',
    'embeddings_index_name': 'word2vec',
    'questions_path': '../data/bg_rc-v1.0.json'
}

# numberbatch_config = {
#     'documents_glob': '../data/context/bgwiki.json/*/wiki_*',
#     'embeddings_path': '../data/conceptnet-numberbatch/bg.txt',
#     'embeddings_index_name': 'numberbatch',
#     'questions_path': '../data/bg_rc-v1.0.json'
# }

In [29]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [3]:
def load_wiki_to_dataframe(documents_glob):
    df = pd.DataFrame()
    for path in glob.glob(documents_glob):
        records = map(ujson.loads, open(path))
        df = df.append(pd.DataFrame.from_records(records))
    df = df.set_index('id')
    df = df.drop('url', axis=1)
    df = df.drop('title', axis=1) # the title is the first sentence inside the text itself
    return df

def fix_utf8_chars(string):
    return ''.join(x for x in string if x.isprintable())

def chunk_document(doc_id, doc_text):
    chunks = Text(fix_utf8_chars(doc_text)).sentences
    return pd.DataFrame.from_dict({
        'id': list(map(lambda i: '{}|{}'.format(doc_id, str(i)), range(len(chunks)))),
        'text': list(map(str, chunks))
    }).set_index('id')

In [4]:
df = load_wiki_to_dataframe(config['documents_glob'])
df

,text
id,
549994,Програма за политическите отношения на сърбо-б...
549996,Кастел дел Монте (село)\n\nКастѐл дел Мо̀нте (...
550003,У дома\n\nУ дома () е американски 3D компютърн...
550011,Нефтохимик 2010\n\nБК „Нефтохимик 2010“ е бълг...
550012,Кървав залез\n\n„Кървав залез“ () е гръцки дра...
550013,"Ноктопластика\n\nНоктопластика е удължаването,..."
550016,Бояна Донева\n\nБояна Георгиева Донева е бълга...
550018,Никола Гандев\n\nНикола Нонев Гандев е българс...
550021,Бернар Лепти\n\nБернар Лепти () е френски исто...


In [7]:
text = df[df.index == '373139']['text'].iloc[0]
text

'Горене\n\nГорене или изгаряне е окислително-редукционен процес, при който се излъчва енергия под формата на топлина и светлина. Горенето също е екзотермична реакция.\n\nКато цяло под понятието горене се разбира окисление на даден материал с кислород под формата на огън. В Химията съществуват реакции под името горене, които се осъществяват без кислород. Такава е реакцията за получаване на флуороводород от флуор и водород. При тази реакция флуорът замества кислорода като окислител.\n\nОтделяне на топлина може да доведе до производството на светлина под формата на тлеене или пламък. Горивата често включват органични съединения (особено въглеводород) в газовата, течната или твърдата фаза.\n\nГоренето е първият химически процес, овладян от човека. Усвояването на огъня изиграва ключова роля в развитието на човешката цивилизация. Огънят открива пред хората възможността за топлинна обработка на храна и отопление на жилищата, а впоследствие\xa0– развитието на металургията, енергетиката и създа

In [ ]:
# doc with id '373139' is interesting

df_chunked = pd.concat(df.progress_apply(lambda x: chunk_document(doc_id=x.name, doc_text=x['text']), axis=1).tolist())
df_chunked

In [12]:
# df_chunked.to_csv('../data/context/df_chunked.csv')
# df_chunked = pd.read_csv('../data/context/df_chunked.csv')
df_chunked.head()

,text
id,
549994|0,Програма за политическите отношения на сърбо-б...
549994|1,Той предвижда създаване на федеративна държава...
549994|2,"Моделът на Австро-Унгария става един пример, к..."
549994|3,На 14 януари 1867 г.
549994|4,"Добродетелната дружина, заедно със сръбския ди..."


In [6]:
# df_chunked[df_chunked.index == '373139|0']

In [13]:
# returns a dict which maps word to embedding vector
def load_embeddings(embeddings_filename):
    embeddings_index = {}

#     with open(embeddings_filename) as f:
    with codecs.open(embeddings_filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    dimensions = len(coefs)
    print('Found {} word vectors of {} dimensions.'.format(len(embeddings_index), dimensions))
    return embeddings_index, dimensions

In [14]:
embeddings_index, embedding_dim = load_embeddings(config['embeddings_path'])

Found 628026 word vectors of 100 dimensions.


In [15]:
def create_vector_space_index(vectors, index_name):
    dimensions = len(vectors[0])
    print('Creating a vector space index of {} vectors of {} dimensions...'.format(len(vectors), dimensions))
    ngtpy.create(index_name, dimensions)
    index = ngtpy.Index(index_name)
    index.batch_insert(vectors)
    index.save()
    print('Vector space index saved as: {}'.format(index_name))
    return index

In [31]:
# vector_index = create_vector_space_index(
#     vectors=list(embeddings_index.values()),
#     index_name=config['embeddings_index_name']
# )

Creating a vector space index of 20870 vectors of 300 dimensions...
Vector space index saved as: numberbatch


In [16]:
def tokenize_to_words(text):
    return list(Text(text).words)

def average_embeddings(word_embeddings, word_embeddings_dim, words):
    embeddings = [
        word_embeddings[word]
        for word in words
        if word in word_embeddings
    ]

    if len(embeddings) > 0:
        return np.average(embeddings, axis=0)
    else:
        return np.zeros(word_embeddings_dim)

def map_text_to_vector(word_embeddings, word_embeddings_dim, text):
    return average_embeddings(word_embeddings, word_embeddings_dim, tokenize_to_words(text))

In [ ]:
# map each text to a vector
df_chunked['vector'] = df_chunked.progress_apply(lambda x: map_text_to_vector(embeddings_index, embedding_dim, x['text']), axis=1)

In [18]:
def find_k_nearest_neighbours(vector_index, query_vector, k=20):
    return vector_index.search(query_vector, k)

In [19]:
# df_chunked.to_csv('../data/context/df_chunked_word2vec.csv')
df_chunked.head()

,text,vector
id,,
549994|0,Програма за политическите отношения на сърбо-б...,"[-0.026610533, -0.1262145, 0.10050154, -0.1033..."
549994|1,Той предвижда създаване на федеративна държава...,"[-0.02348755, -0.11426931, 0.18817468, -0.1660..."
549994|2,"Моделът на Австро-Унгария става един пример, к...","[0.039585475, -0.11630816, 0.15598468, -0.1696..."
549994|3,На 14 януари 1867 г.,"[0.07213081, -0.167631, 0.393249, -0.1446622, ..."
549994|4,"Добродетелната дружина, заедно със сръбския ди...","[-0.049288053, -0.107488096, 0.08172013, -0.12..."


In [20]:
vector_index = create_vector_space_index(
    vectors=df_chunked['vector'].tolist(),
    index_name=config['embeddings_index_name']
)

Creating a vector space index of 2502559 vectors of 100 dimensions...
Vector space index saved as: word2vec


In [21]:
import importlib
import sys
sys.path.insert(0, '..')
import dataset

from dataset.bgquiz import BGQuiz
_ = importlib.reload(dataset.bgquiz)

In [51]:
def augment_dataset(path):
    counter = 0
    with open(path, 'r') as f:
        json_data = json.load(f)

        for (category, items) in json_data['data'].items():
            for questions in items:
                for q in questions['questions']:
                    query_texts = [q['question']] + [answer for answer in q['answers']]
                    
                    query_vectors = [
                        map_text_to_vector(embeddings_index, embedding_dim, query_text)
                        for query_text in query_texts
                    ]
                    
                    nearest_vectors = sorted(flatten([
                        find_k_nearest_neighbours(vector_index, query_vector)
                        for query_vector in query_vectors
                    ]), key=lambda x: x[1])[:30]
                    
                    most_relevant_texts = [
                        df_chunked.iloc[index]['text']
                        for index, distance in nearest_vectors
                    ]
                    
                    q['context'] = most_relevant_texts
                    
                    counter = counter + 1
                    if counter % 100 == 0:
                        print('Processed {} questions.'.format(counter))
        return json_data

In [ ]:
augmented_json = augment_dataset(config['questions_path'])

In [53]:
with open('../data/bg_rc_context-v1.0.json', 'w', encoding='utf-8') as outfile:
    json.dump(augmented_json, outfile, ensure_ascii=False, indent=2)

In [22]:
quiz = BGQuiz(config['questions_path'])

In [23]:
for question in quiz.iterator():
    print(question.question, question.answers)
    query_text = question.answers[2]
    question_vector = map_text_to_vector(embeddings_index, embedding_dim, query_text)
    result = find_k_nearest_neighbours(vector_index, question_vector)
    print(result)
    for index, distance in result:
        print(df_chunked.iloc[index]['text'])
    break

Detector is not able to detect the language reliably.


Самостоятелно съществуващи живи системи са: ['вирусите', 'тъканите', 'митохондриите', 'едноклетъчните организми']
[(2082228, 2.3715546131134033), (2104201, 2.381986379623413), (996166, 2.3974997997283936), (1546897, 2.46706485748291), (232713, 2.4701201915740967), (1458622, 2.5402672290802), (1205206, 2.5462875366210938), (462492, 2.566333532333374), (2363684, 2.574266195297241), (1646099, 2.575312376022339), (1520336, 2.582143545150757), (1496028, 2.5882468223571777), (1581737, 2.592104196548462), (1785458, 2.592341899871826), (179863, 2.593149423599243), (1612752, 2.6007392406463623), (1025001, 2.604177713394165), (1129323, 2.6057627201080322), (1785357, 2.6119003295898438), (1360583, 2.61767578125)]
Вирусният ензим неураминидаза разрушава мукополизахаридите на клетъчната мембрана на клетката гостоприемник.
Всяка молекула хемоглобин свързва 4 молекули кислород.
Всички гени в нуклеиновата киселина изграждат вирусния геном.
Към органичните вещества спадат плазмените белтъци.
Тези клетк

In [31]:
vector_index.get_object(583552)

[0.18894624710083008,
 -0.2666137218475342,
 0.2841305136680603,
 -0.3475459814071655,
 0.385382741689682,
 -0.1279895007610321,
 -0.15750449895858765,
 -0.2708809971809387,
 -0.24386675655841827,
 0.2688262462615967,
 0.42385149002075195,
 0.39767327904701233,
 -0.40571150183677673,
 -0.3425554931163788,
 -0.27159225940704346,
 0.27097299695014954,
 -0.04822725057601929,
 -0.05412774905562401,
 0.09443475306034088,
 -0.14338752627372742,
 0.000898752361536026,
 0.10135675221681595,
 0.25765374302864075,
 -0.06950050592422485,
 0.2156815081834793,
 0.01806749403476715,
 0.22434525191783905,
 0.046958498656749725,
 -0.1141357421875,
 0.21112149953842163,
 0.08602000027894974,
 -0.19362349808216095,
 -0.33401399850845337,
 0.08598874509334564,
 0.19498424232006073,
 0.1352977454662323,
 0.026406999677419662,
 -0.023005999624729156,
 0.14930924773216248,
 -0.284455269575119,
 -0.13345524668693542,
 0.18775376677513123,
 -0.11257950216531754,
 -0.10299549996852875,
 -0.06228775158524513,
 

In [33]:
df_chunked.iloc[583552]['text']

'Основните и компоненти са:'